# Working with Data Sets


In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
Films=pd.read_csv('datasets\movies.csv', sep=',',decimal=".")

In [4]:
Films.head()

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...
1,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...
2,3,Inception,2010,8.8,74.0,"Action, Adventure, Sci-Fi",1760209,Christopher Nolan,148,292.58,A thief who steals corporate secrets through t...
3,4,Fight Club,1999,8.8,66.0,Drama,1609459,David Fincher,139,37.03,An insomniac office worker and a devil-may-car...
4,5,Pulp Fiction,1994,8.9,94.0,"Crime, Drama",1570194,Quentin Tarantino,154,107.93,"The lives of two mob hitmen, a boxer, a gangst..."


In [3]:
Oscar=pd.read_csv('datasets/the_oscar_award.csv')

In [5]:
Oscar.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [8]:
Befta=pd.read_csv('datasets/bafta_films.csv')
Befta.head()

,year,category,nominee,workers,winner
0,1949,Film | British Film in 1949,The Fallen Idol,NaN,True
1,1949,Film | British Film in 1949,Hamlet,NaN,False
2,1949,Film | British Film in 1949,Oliver Twist,NaN,False
3,1949,Film | British Film in 1949,Once A Jolly Swagman,NaN,False
4,1949,Film | British Film in 1949,The Red Shoes,NaN,False


In [9]:
Globe=pd.read_csv('datasets/golden_globe_awards.csv')
Globe.head()

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


### Первый данные с фильмами не достаточны для нас, так как там нет записи об актерах, которые нужны для оценки популярности фильма


In [68]:
oscartable = pd.merge(left=Films, right=Oscar[['year_film', 'film','winner']], left_on= ['Title','Year'], right_on =['film','year_film'], how='left')
oscartable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13521 entries, 0 to 13520
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         13521 non-null  int64  
 1   Title        13521 non-null  object 
 2   Year         13521 non-null  int64  
 3   Score        13521 non-null  float64
 4   Metascore    9253 non-null   float64
 5   Genre        13521 non-null  object 
 6   Vote         13521 non-null  int64  
 7   Director     13520 non-null  object 
 8   Runtime      13521 non-null  int64  
 9   Revenue      10359 non-null  float64
 10  Description  13521 non-null  object 
 11  year_film    5030 non-null   float64
 12  film         5030 non-null   object 
 13  winner       5030 non-null   object 
dtypes: float64(4), int64(4), object(6)
memory usage: 1.5+ MB


In [69]:
# заменяем на 1 и 0 в колонке winner
d = {False : 0, True : 1}
oscartable['winner'] = oscartable['winner'].map(d)

#находим индексы нулевых строк которые нужно заполнить
all_indexes = oscartable[oscartable.winner.isnull()].index
oscartable.loc[all_indexes, 'year_film':'winner'] = -1

#удаляем ненужные столбцы
oscartable.drop(['year_film', 'film'], axis='columns', inplace=True)
#называем winner в oscar
oscartable.rename(columns={'winner': 'Oscar'}, inplace=True)

In [70]:
# заполняем нулевые значения в других колонках 
metascore_indexes = oscartable[oscartable.Metascore.isnull()].index
oscartable.loc[metascore_indexes, 'Metascore'] = -1
revenue_indexes = oscartable[oscartable.Revenue.isnull()].index
oscartable.loc[revenue_indexes, 'Revenue'] = -1
# у одного фильма не указан режисер, помечаем это
director_indexes = oscartable[oscartable.Director.isnull()].index
oscartable.loc[director_indexes, 'Director'] = 'No info'
oscartable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13521 entries, 0 to 13520
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         13521 non-null  int64  
 1   Title        13521 non-null  object 
 2   Year         13521 non-null  int64  
 3   Score        13521 non-null  float64
 4   Metascore    13521 non-null  float64
 5   Genre        13521 non-null  object 
 6   Vote         13521 non-null  int64  
 7   Director     13521 non-null  object 
 8   Runtime      13521 non-null  int64  
 9   Revenue      13521 non-null  float64
 10  Description  13521 non-null  object 
 11  Oscar        13521 non-null  float64
dtypes: float64(4), int64(4), object(4)
memory usage: 2.0+ MB


In [71]:
# сортируем по названию и победил/номинант/не номинант, для сортировки
oscartable=oscartable.sort_values(by=['Title', 'Oscar'], ascending=[True, False])
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
#удаляем дубли из-за разных номинацихй
oscartable.drop_duplicates(subset=['Title', 'Year'], 
                     keep = 'first', inplace = True)
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
fulltable = oscartable
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 5348 to 8512
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         9999 non-null   int64  
 1   Title        9999 non-null   object 
 2   Year         9999 non-null   int64  
 3   Score        9999 non-null   float64
 4   Metascore    9999 non-null   float64
 5   Genre        9999 non-null   object 
 6   Vote         9999 non-null   int64  
 7   Director     9999 non-null   object 
 8   Runtime      9999 non-null   int64  
 9   Revenue      9999 non-null   float64
 10  Description  9999 non-null   object 
 11  Oscar        9999 non-null   float64
dtypes: float64(4), int64(4), object(4)
memory usage: 1015.5+ KB


## Делаем тоже самое для других премий 

In [72]:
beftatable = pd.merge(left=fulltable, right=Befta[[ 'nominee','winner']], left_on= ['Title'], right_on =['nominee'], how='left')
beftatable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 10694
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         10695 non-null  int64  
 1   Title        10695 non-null  object 
 2   Year         10695 non-null  int64  
 3   Score        10695 non-null  float64
 4   Metascore    10695 non-null  float64
 5   Genre        10695 non-null  object 
 6   Vote         10695 non-null  int64  
 7   Director     10695 non-null  object 
 8   Runtime      10695 non-null  int64  
 9   Revenue      10695 non-null  float64
 10  Description  10695 non-null  object 
 11  Oscar        10695 non-null  float64
 12  nominee      1149 non-null   object 
 13  winner       1149 non-null   object 
dtypes: float64(4), int64(4), object(6)
memory usage: 1.2+ MB


In [73]:
# заменяем на 1 и 0 в колонке winner
d = {False : 0, True : 1}
beftatable['winner'] = beftatable['winner'].map(d)
#удаляем ненужные столбцы
beftatable.drop(['nominee'], axis='columns', inplace=True)

#находим индексы нулевых строк которые нужно заполнить
all_indexes = beftatable[beftatable.winner.isnull()].index
beftatable.loc[all_indexes, 'winner'] = -1

#называем winner в Befta
beftatable.rename(columns={'winner': 'Befta'}, inplace=True)
beftatable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 10694
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         10695 non-null  int64  
 1   Title        10695 non-null  object 
 2   Year         10695 non-null  int64  
 3   Score        10695 non-null  float64
 4   Metascore    10695 non-null  float64
 5   Genre        10695 non-null  object 
 6   Vote         10695 non-null  int64  
 7   Director     10695 non-null  object 
 8   Runtime      10695 non-null  int64  
 9   Revenue      10695 non-null  float64
 10  Description  10695 non-null  object 
 11  Oscar        10695 non-null  float64
 12  Befta        10695 non-null  float64
dtypes: float64(5), int64(4), object(4)
memory usage: 1.1+ MB


In [74]:
# сортируем по названию и победил/номинант/не номинант, для сортировки
beftatable=beftatable.sort_values(by=['Title', 'Befta'], ascending=[True, False])
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
#удаляем дубли из-за разных номинаций
beftatable.drop_duplicates(subset =['Title', 'Year'], 
                     keep = 'first', inplace = True)
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
fulltable = beftatable
#должно быть 9699
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 10694
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         9999 non-null   int64  
 1   Title        9999 non-null   object 
 2   Year         9999 non-null   int64  
 3   Score        9999 non-null   float64
 4   Metascore    9999 non-null   float64
 5   Genre        9999 non-null   object 
 6   Vote         9999 non-null   int64  
 7   Director     9999 non-null   object 
 8   Runtime      9999 non-null   int64  
 9   Revenue      9999 non-null   float64
 10  Description  9999 non-null   object 
 11  Oscar        9999 non-null   float64
 12  Befta        9999 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 1.1+ MB


In [75]:
globetable = pd.merge(left=fulltable, right=Globe[['year_film', 'film','win']], left_on= ['Title','Year'], right_on =['film','year_film'], how='left')
globetable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11025 entries, 0 to 11024
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         11025 non-null  int64  
 1   Title        11025 non-null  object 
 2   Year         11025 non-null  int64  
 3   Score        11025 non-null  float64
 4   Metascore    11025 non-null  float64
 5   Genre        11025 non-null  object 
 6   Vote         11025 non-null  int64  
 7   Director     11025 non-null  object 
 8   Runtime      11025 non-null  int64  
 9   Revenue      11025 non-null  float64
 10  Description  11025 non-null  object 
 11  Oscar        11025 non-null  float64
 12  Befta        11025 non-null  float64
 13  year_film    1916 non-null   float64
 14  film         1916 non-null   object 
 15  win          1916 non-null   object 
dtypes: float64(6), int64(4), object(6)
memory usage: 1.4+ MB


In [76]:
# заменяем на 1 и 0 в колонке win
d = {False : 0, True : 1}
globetable['win'] = globetable['win'].map(d)
#удаляем ненужные столбцы
globetable.drop(['year_film','film'], axis='columns', inplace=True)

#находим индексы нулевых строк которые нужно заполнить
all_indexes = globetable[globetable.win.isnull()].index
globetable.loc[all_indexes, 'win'] = -1

#называем winner в Befta
globetable.rename(columns={'win': 'Globe'}, inplace=True)
globetable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11025 entries, 0 to 11024
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         11025 non-null  int64  
 1   Title        11025 non-null  object 
 2   Year         11025 non-null  int64  
 3   Score        11025 non-null  float64
 4   Metascore    11025 non-null  float64
 5   Genre        11025 non-null  object 
 6   Vote         11025 non-null  int64  
 7   Director     11025 non-null  object 
 8   Runtime      11025 non-null  int64  
 9   Revenue      11025 non-null  float64
 10  Description  11025 non-null  object 
 11  Oscar        11025 non-null  float64
 12  Befta        11025 non-null  float64
 13  Globe        11025 non-null  float64
dtypes: float64(6), int64(4), object(4)
memory usage: 1.3+ MB


In [77]:
# сортируем по названию и победил/номинант/не номинант, для сортировки
globetable=globetable.sort_values(by=['Title', 'Globe'], ascending=[True, False])
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
#удаляем дубли из-за разных номинаций
globetable.drop_duplicates(subset =['Title','Year'], 
                     keep = 'first', inplace = True)
#тестовая проверка
#oscartable[oscartable['Title'] == 'The Dark Knight']
fulltable = globetable
#должно быть 9699
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 11024
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         9999 non-null   int64  
 1   Title        9999 non-null   object 
 2   Year         9999 non-null   int64  
 3   Score        9999 non-null   float64
 4   Metascore    9999 non-null   float64
 5   Genre        9999 non-null   object 
 6   Vote         9999 non-null   int64  
 7   Director     9999 non-null   object 
 8   Runtime      9999 non-null   int64  
 9   Revenue      9999 non-null   float64
 10  Description  9999 non-null   object 
 11  Oscar        9999 non-null   float64
 12  Befta        9999 non-null   float64
 13  Globe        9999 non-null   float64
dtypes: float64(6), int64(4), object(4)
memory usage: 1.1+ MB


In [78]:
fulltable.loc[:, 'win'] = 0
fulltable['win'] = fulltable[['Oscar','Befta','Globe']].max(axis =1)
fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 11024
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         9999 non-null   int64  
 1   Title        9999 non-null   object 
 2   Year         9999 non-null   int64  
 3   Score        9999 non-null   float64
 4   Metascore    9999 non-null   float64
 5   Genre        9999 non-null   object 
 6   Vote         9999 non-null   int64  
 7   Director     9999 non-null   object 
 8   Runtime      9999 non-null   int64  
 9   Revenue      9999 non-null   float64
 10  Description  9999 non-null   object 
 11  Oscar        9999 non-null   float64
 12  Befta        9999 non-null   float64
 13  Globe        9999 non-null   float64
 14  win          9999 non-null   float64
dtypes: float64(7), int64(4), object(4)
memory usage: 1.2+ MB


In [81]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(fulltable, test_size=0.3)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 7911 to 9896
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         3000 non-null   int64  
 1   Title        3000 non-null   object 
 2   Year         3000 non-null   int64  
 3   Score        3000 non-null   float64
 4   Metascore    3000 non-null   float64
 5   Genre        3000 non-null   object 
 6   Vote         3000 non-null   int64  
 7   Director     3000 non-null   object 
 8   Runtime      3000 non-null   int64  
 9   Revenue      3000 non-null   float64
 10  Description  3000 non-null   object 
 11  Oscar        3000 non-null   float64
 12  Befta        3000 non-null   float64
 13  Globe        3000 non-null   float64
 14  win          3000 non-null   float64
dtypes: float64(7), int64(4), object(4)
memory usage: 375.0+ KB


In [82]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
X=train[['Score', 'Revenue', 'Vote']]
y=train.win
#X=train['Score']
#y=train.winner
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [83]:
expected = test.win
predicted = model.predict(test[['Score', 'Revenue', 'Vote']])
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted,labels=[1,0,-1]))

              precision    recall  f1-score   support

        -1.0       0.83      0.99      0.90      2452
         0.0       0.28      0.02      0.04       320
         1.0       0.65      0.06      0.10       228

    accuracy                           0.82      3000
   macro avg       0.59      0.36      0.35      3000
weighted avg       0.75      0.82      0.75      3000

[[  13    7  208]
 [   5    7  308]
 [   2   11 2439]]


In [84]:
print(fulltable[:5].to_csv())

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,Oscar,Befta,Globe,win
0,3166,'71,2014,7.2,83.0,"Action, Drama, Thriller",44532,Yann Demange,99,1.27,"In 1971, a young and disorientated British soldier is accidentally abandoned by his unit following a riot on the deadly streets of Belfast.",-1.0,-1.0,-1.0,-1.0
1,6646,'A' gai wak,1983,7.4,-1.0,"Action, Comedy",12657,Jackie Chan,105,-1.0,"Fighting against pirates in ""old Hong Kong"". Chinese costume drama with plenty of over-the-top tongue in cheek action and music.",-1.0,-1.0,-1.0,-1.0
2,8656,'A' gai wak 2,1987,7.2,-1.0,"Action, Comedy, Crime",7739,Jackie Chan,101,-1.0,"Dragon is now transferred to be the police head of Sai Wan district, and has to contend with a gangster kingpin, anti-Manchu revolutionaries, some runaway pirates, Manchu Loyalists and a corrupt Police Superintendent.",-1.0,-1.0,-1.0,-1.0
3,9287,'I Know Where I'm Going!',1945,7.6,-1.0,"Drama, Romance",6861,Michael Powell,91,-1.0,"A young E

In [94]:
fulltable.to_csv (r'data_new.csv', index = False)